# CNN using pytorch

### CIFAR-10 dataset

In [2]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms 
import matplotlib.pyplot as plt
import torch.nn.functional as F
import numpy as np

In [3]:
# device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
# Hyper-parameters
num_epochs = 4
batch_size = 4
learning_rate = 0.001

In [5]:
# dataset has PILImage images of range [0, 1].
# We transform them to Tensors of normalized range [-1, 1]
transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [7]:
train_dataset = torchvision.datasets.CIFAR10(
    root='./data', 
    train=True, 
    download=True, 
    transform=transforms)

test_dataset = torchvision.datasets.CIFAR10(
    root='./data', 
    train=False, 
    download=True, 
    transform=transforms)

train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=batch_size, 
    shuffle=False)

100.0%


Extracting ./data\cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [8]:
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [18]:
# implement the CNN

class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.pool = nn.MaxPool2d(2, 2)
        
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        # 16 * 5 * 5 is the size of the input to the fully connected layer
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    
    def forward(self, x):
        
        # first conv2d layer
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool(x)
        
        
        # second conv2d layer
        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool(x)
        
        # flatten the tensor
        x = x.view(-1, 16 * 5 * 5)
        
        
        # first fully connected layer
        x = self.fc1(x)
        x = F.relu(x)
        
        
        # second fully connected layer  
        x = self.fc2(x)
        x = F.relu(x)
        
        
        # third fully connected layer
        x = self.fc3(x)
        # last is softmax layer but that is done by optimizer
        
        return x
        

In [19]:
model = ConvNet().to(device)

In [20]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [21]:
n_total_steps = len(train_loader)

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # original shap: [4,3,32,32] = 4,3,1024
        # input_layer: 3 input channels, 6 output channels, 5 kernel size
        
        images = images.to(device)
        labels = labels.to(device)
        
        # forward pass
        output = model(images)
        loss = criterion(output, labels)
        
        # backward pass and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i + 1) % 2000 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item()}')
            
print('Finished Training')

Epoch [1/4], Step [2000/12500], Loss: 2.2733073234558105
Epoch [1/4], Step [4000/12500], Loss: 2.2589821815490723
Epoch [1/4], Step [6000/12500], Loss: 2.291543960571289
Epoch [1/4], Step [8000/12500], Loss: 2.2939717769622803
Epoch [1/4], Step [10000/12500], Loss: 2.211657762527466
Epoch [1/4], Step [12000/12500], Loss: 1.9885542392730713
Epoch [2/4], Step [2000/12500], Loss: 2.1269664764404297
Epoch [2/4], Step [4000/12500], Loss: 2.101356029510498
Epoch [2/4], Step [6000/12500], Loss: 1.8249123096466064
Epoch [2/4], Step [8000/12500], Loss: 1.6811940670013428
Epoch [2/4], Step [10000/12500], Loss: 2.2015883922576904
Epoch [2/4], Step [12000/12500], Loss: 2.237572193145752
Epoch [3/4], Step [2000/12500], Loss: 1.8212952613830566
Epoch [3/4], Step [4000/12500], Loss: 2.131331205368042
Epoch [3/4], Step [6000/12500], Loss: 1.9895493984222412
Epoch [3/4], Step [8000/12500], Loss: 1.1224913597106934
Epoch [3/4], Step [10000/12500], Loss: 1.3096091747283936
Epoch [3/4], Step [12000/12500]

In [22]:
with torch.no_grad():
    n_samples = 0
    n_correct = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()
        
        for i in range(batch_size):
            label = labels[i]
            pred = predicted[i]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1
            
    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {acc} %')
    
    for i in range(10):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'Accuracy of {classes[i]}: {acc} %')

Accuracy of the network: 44.31 %
Accuracy of plane: 51.6 %
Accuracy of car: 38.7 %
Accuracy of bird: 34.0 %
Accuracy of cat: 29.8 %
Accuracy of deer: 27.3 %
Accuracy of dog: 38.7 %
Accuracy of frog: 56.0 %
Accuracy of horse: 59.3 %
Accuracy of ship: 58.4 %
Accuracy of truck: 49.3 %
